In [7]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
pd.set_option('display.max_columns', None)
import warnings
from scrape_stats_function import scrape_league_data



years = ["2024-2025","2023-2024","2022-2023","2021-2022","2020-2021","2019-2020","2018-2019","2017-2018","2016-2017","2015-2016","2014-2015","2013-2014","2012-2013","2011-2012","2010-2011"]
leagueinfo = [
    {
        "name": "Bundesliga",
        "url": "https://fbref.com/en/comps/20/{year}/schedule/{year}-Bundesliga-Scores-and-Fixtures"
    },
    {
        "name": "Premier League",
        "url": "https://fbref.com/en/comps/9/{year}/schedule/{year}-Premier-League-Scores-and-Fixtures"
    },
    {
        "name": "Serie A",
        "url": "https://fbref.com/en/comps/11/{year}/schedule/{year}-Serie-A-Scores-and-Fixtures"
    },
    {
        "name": "La Liga",
        "url": "https://fbref.com/en/comps/12/{year}/schedule/{year}-La-Liga-Scores-and-Fixtures"
    },
    {
        "name": "Ligue 1",
        "url": "https://fbref.com/en/comps/13/{year}/schedule/{year}-Ligue-1-Scores-and-Fixtures"
    },
    {
        "name": "Eredivisie",
        "url": "https://fbref.com/en/comps/23/{year}/schedule/{year}-Eredivisie-Scores-and-Fixtures"
    },
    {        "name": "Primeira Liga",
        "url": "https://fbref.com/en/comps/32/{year}/schedule/{year}-Primeira-Liga-Scores-and-Fixtures"
    
    },
    {
        "name": "EFL Championship",
        "url": "https://fbref.com/en/comps/10/{year}/schedule/{year}-EFL-Championship-Scores-and-Fixtures"
    },
    {
        "name": "Scottish Premiership",
        "url": "https://fbref.com/en/comps/40/{year}/schedule/{year}-Scottish-Premiership-Scores-and-Fixtures"
    }
]


leagues = []
for year in years:
    for league in leagueinfo:
        leagues.append({
            "name": f"{league['name']} {year}",
            "url": league["url"].format(year=year),
        })

league_dfs = {}
combined_list = []
for league in leagues:
    df = scrape_league_data(league["name"], league["url"], table_id=None)
    if df is not None:
        league_dfs[league["name"]] = df
        combined_list.append(df)
    else:
        print(f"Failed for {league['name']} at {league['url']}")
    time.sleep(5) 

ImportError: cannot import name 'scrape_league_data' from 'scrape_stats_function' (c:\Users\JMerilien\OneDrive - Lennar Azure AD\Documents\side projects\football-stats-webscrap\scrape_stats_function.py)

In [4]:
df2 = pd.concat(combined_list,ignore_index=True)
df2 = df2.drop(['Notes','Match Report','Round'], axis=1)

df2.columns

int64_col = ['Wk','Attendance']
float64_col = ['xG','xG.1']


for col in int64_col:
    df2[col] = pd.to_numeric(df2[col], errors='coerce').astype('Int64')
for col in float64_col:
    df2[col] = pd.to_numeric(df2[col], errors='coerce').astype('float64')

df2 = df2.astype({
    'Day': 'string',
    'Home': 'string',
    'Away': 'string',
    'Venue': 'string',
    'Referee': 'string',
    'League': 'string',
    'Score': 'string',
    'Date':'string',
    'Time':'string'

})
df2.dtypes

Wk                     Int64
Day           string[python]
Date          string[python]
Time          string[python]
Home          string[python]
xG                   float64
Score         string[python]
xG.1                 float64
Away          string[python]
Attendance             Int64
Venue         string[python]
Referee       string[python]
League        string[python]
dtype: object

In [1]:
new_columns = {
    'xG': 'Home xG',
    'xG.1':'Away xG',
}

df2.rename(columns=new_columns, inplace=True)

df2

NameError: name 'df2' is not defined